In [1]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import numpy as np

In [2]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib2.urlopen(url)
page

In [3]:
soup = BeautifulSoup(page,'html.parser')
table = soup.find("table",{"class":"wikitable sortable"}) #.wikitable wikitable sortable jquery-tablesorter


Extracting data from table and converting to dataframe

In [4]:
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
    
data = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])
data['Neighbourhood'] = data.Neighbourhood.str.strip()
data.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Formatting data as per requirement
1. Remove 'Not assigned Boroughs'
2. Imputing 'Not assigned' Neighbourhood.
3. Grouping data based on Borough.

In [5]:
data.dropna(how='all', inplace = True)
data = data[data.Borough != 'Not assigned']

data['Neighbourhood'] = data.apply(lambda x: x['Borough'] if x['Neighbourhood']=='Not assigned'  else x['Neighbourhood'], axis = 1 )
data.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [6]:
data['Neighbourhood'] = data.groupby(['Postcode','Borough']).transform(lambda x: ','.join(x))
data = data.drop_duplicates()
data.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Harbourfront,Regent Park"
7,M6A,North York,"Lawrence Heights,Lawrence Manor"
9,M7A,Queen's Park,Queen's Park


In [7]:
data.shape

(103, 3)